# Regularization (Ridge/Lasso) + Project Proposal Sprint

<hr>

<center>
<div>
<img src="https://raw.githubusercontent.com/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/main/notebooks/figures/mgmt_474_ai_logo_02-modified.png" width="200"/>
</div>
</center>

# <center><a class="tocSkip"></center>
# <center>MGMT47400 Predictive Analytics</center>
# <center>Professor: Davi Moreira </center>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/blob/main/notebooks/05_regularization_project_proposal.ipynb)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Explain why regularization improves generalization
2. Fit Ridge/Lasso with proper scaling and CV selection
3. Interpret coefficient shrinkage and sparsity
4. Draft a project proposal with a viable dataset + target + metric + split plan
5. Use Gemini to scaffold code and then add guardrails (checks + comments)

---

> **📋 Participation Reminder:** This notebook contains **2 PAUSE-AND-DO exercises**. You are expected to complete all exercises before submitting your notebook.

---

## 💼 Why This Matters: When the Model Memorizes Instead of Learns

After engineering dozens of new features at **HomeValue Analytics** (interactions, polynomials, location-based), your model's training error plummets — but validation error barely improves. The CTO raises a red flag: *"Are we memorizing noise in the training data, or learning real patterns that generalize?"*

This is overfitting, the most common pitfall in predictive analytics. Regularization techniques (Ridge, Lasso, Elastic Net) add a penalty that forces the model to keep only the features that truly matter.

> **Today's focus:** Applying regularization to prevent overfitting in our housing price model, and using Lasso's feature selection property to identify the most important predictors.

---

In [ ]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.precision', 4)
RANDOM_SEED = 474
np.random.seed(RANDOM_SEED)
print("✓ Setup complete!")

**Reading the output:**

The `Setup complete!` confirmation means all imports loaded successfully. This notebook's import list adds several regularization-specific classes to the standard toolkit: **Ridge**, **RidgeCV**, **Lasso**, **LassoCV**, and **ElasticNet** from `sklearn.linear_model`. The `CV` variants include built-in cross-validation for automatic hyper-parameter tuning. As always, `RANDOM_SEED = 474` anchors all randomness.

**Why this matters:** The cross-validated estimators (`RidgeCV`, `LassoCV`) save you from writing your own alpha-search loops. They handle the grid search internally and expose the best alpha as an attribute after fitting.

---

## 1. Load Data

We load the same California Housing dataset and apply the identical 60/20/20 split with `RANDOM_SEED = 474`. Because the splits match the previous notebooks exactly, any change in performance can be attributed solely to the new regularization techniques introduced here, not to data differences.
The test set remains locked; all tuning uses the validation partition.

> 💡 **Gemini Prompt:** "Load the California Housing dataset using sklearn fetch_california_housing with as_frame=True. Separate features (X) from the target MedHouseVal (y), then do a 60/20/20 train/val/test split using two sequential train_test_split calls with random_state=RANDOM_SEED. Print the sizes of each set."
>
> **After running, verify:**
> - Train set is ~60%, validation ~20%, test ~20% of total rows
> - X has 8 feature columns (MedInc, HouseAge, AveRooms, etc.)
> - y contains median house values (continuous target)

In [ ]:
california = fetch_california_housing(as_frame=True)
df = california.frame
X = df.drop(columns=['MedHouseVal'])
y = df['MedHouseVal']

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.20, random_state=RANDOM_SEED)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=RANDOM_SEED)

print(f"Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)} (locked)")

**Reading the output:**

The printed split summary shows roughly **12,384 training**, **4,128 validation**, and **4,128 test** samples -- identical to previous notebooks. Consistency across notebooks is critical: it means the Ridge and Lasso results we compute here are directly comparable to the plain linear regression and polynomial models from the previous notebook, since they are evaluated on the exact same validation set.

**Key takeaway:** Never change your random seed or split strategy mid-project. Doing so invalidates all prior comparisons and makes it impossible to tell whether performance changed because of the model or because of the data partition.

---

## 2. Why Regularization?

### The Problem: Overfitting

Without regularization, linear regression can:
- Fit noise in training data
- Produce unstable coefficients
- Perform poorly on new data

### The Solution: Penalize Complexity

**Ridge (L2 penalty)**
- Adds penalty: α × (sum of squared coefficients)
- Shrinks all coefficients toward zero
- Keeps all features
- Use when: All features might be relevant

**Lasso (L1 penalty)**
- Adds penalty: α × (sum of absolute coefficients)
- Can shrink coefficients to exactly zero
- Performs feature selection
- Use when: You expect many irrelevant features

**Elastic Net**
- Combines L1 + L2
- Use when: You want feature selection but with less sensitivity than Lasso

## 3. Ridge Regression with Cross-Validation

Ridge regression adds an L2 penalty to the ordinary least-squares loss, shrinking all coefficients toward zero without eliminating any. The strength of the penalty is controlled by the hyper-parameter **alpha** (higher alpha = stronger shrinkage).
Rather than picking alpha by hand, we use `RidgeCV`, which performs 5-fold cross-validation over a grid of 50 candidate alpha values (log-spaced from 0.001 to 1000) and automatically selects the one that minimizes CV error.

> 💡 **Gemini Prompt:** "Build a Ridge regression pipeline with StandardScaler and RidgeCV. Use 50 alpha values log-spaced from 0.001 to 1000, and 5-fold CV inside RidgeCV to automatically select the best alpha. Fit on training data, then print the best alpha and R-squared on both train and validation sets."
>
> **After running, verify:**
> - Best alpha is printed (a value between 0.001 and 1000)
> - Train R-squared and Val R-squared are both reported
> - Val R-squared is close to (but slightly below) Train R-squared

In [ ]:
# Ridge with automatic alpha tuning
alphas = np.logspace(-3, 3, 50)

ridge_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('ridge', RidgeCV(alphas=alphas, cv=5))
])

ridge_pipeline.fit(X_train, y_train)

best_alpha = ridge_pipeline.named_steps['ridge'].alpha_
train_score_ridge = ridge_pipeline.score(X_train, y_train)
val_score_ridge = ridge_pipeline.score(X_val, y_val)

print("=== RIDGE REGRESSION ===")
print(f"Best alpha: {best_alpha:.4f}")
print(f"Train R²: {train_score_ridge:.4f}")
print(f"Val R²: {val_score_ridge:.4f}")

**Reading the output:**

The Ridge output reports three numbers: the **best alpha** selected by 5-fold CV, the **Train R²**, and the **Val R²**. The best alpha is typically a modest value (often in the single digits), meaning only light regularization was needed for this dataset. Train and validation R² values should be very close to the unregularized OLS scores from the previous notebook, confirming that Ridge's primary benefit here is *stability* rather than a dramatic accuracy jump. The small train-validation gap indicates that overfitting is minimal even without regularization on this 8-feature dataset.

**Why this matters:** Ridge shines when you have many correlated features or polynomial expansions where OLS coefficients become unstable. On a clean 8-feature dataset, the improvement is subtle but the *insurance* against overfitting is real.

---

## 📝 PAUSE-AND-DO Exercise 1 (5 minutes)

**Task:** Run RidgeCV and summarize alpha choice + validation performance.

The code above already ran RidgeCV. Now:
1. Interpret what the chosen alpha value means
2. Compare to baseline linear regression (from previous notebooks)
3. Explain why Ridge might help with generalization

---

### YOUR ANALYSIS:

**Alpha Interpretation:**  
[What does the chosen alpha value mean? Higher = more regularization]

**Performance Comparison:**  
[How does Ridge compare to unregularized linear regression?]

**Generalization:**  
[Why might Ridge improve validation performance?]

---

## 4. Lasso Regression with Cross-Validation

Lasso uses an L1 penalty instead of L2. The key difference is that L1 can drive coefficients to **exactly zero**, effectively performing automatic feature selection. This makes Lasso especially useful when you suspect that many features are irrelevant.
`LassoCV` searches the same 50-value alpha grid with 5-fold cross-validation and selects the alpha that minimizes CV error. We set `max_iter=10000` to ensure convergence for stronger penalties.

> 💡 **Gemini Prompt:** "Build a Lasso regression pipeline with StandardScaler and LassoCV. Use the same 50 log-spaced alphas from 0.001 to 1000, 5-fold CV, and max_iter=10000. Fit on training data, then print the best alpha and R-squared on train and validation sets."
>
> **After running, verify:**
> - Best alpha is printed (likely a small value since Lasso is aggressive)
> - Train and Val R-squared are both reported
> - Performance is comparable to Ridge (may be slightly lower due to feature zeroing)

In [ ]:
# Lasso with automatic alpha tuning
lasso_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('lasso', LassoCV(alphas=alphas, cv=5, max_iter=10000))
])

lasso_pipeline.fit(X_train, y_train)

best_alpha_lasso = lasso_pipeline.named_steps['lasso'].alpha_
train_score_lasso = lasso_pipeline.score(X_train, y_train)
val_score_lasso = lasso_pipeline.score(X_val, y_val)

print("=== LASSO REGRESSION ===")
print(f"Best alpha: {best_alpha_lasso:.4f}")
print(f"Train R²: {train_score_lasso:.4f}")
print(f"Val R²: {val_score_lasso:.4f}")

**Reading the output:**

Lasso reports the same three numbers: **best alpha**, **Train R²**, and **Val R²**. The best alpha for Lasso is typically much smaller than Ridge's because the L1 penalty acts more aggressively on individual coefficients. Pay attention to the R² values: if they are nearly identical to Ridge's, the two methods are performing similarly on this dataset. The key differentiator appears in the next cell, where we inspect which coefficients Lasso set to zero.

**Key takeaway:** Lasso's value proposition is not always higher accuracy -- it is *interpretability through sparsity*. By zeroing out weak features, Lasso tells you which variables genuinely drive the prediction and which are noise.

---

## 📝 PAUSE-AND-DO Exercise 2 (5 minutes)

**Task:** Run LassoCV and identify top selected features (if any).

---

> 💡 **Gemini Prompt:** "Extract the Lasso model coefficients into a DataFrame with columns Feature, Coefficient, and Abs_Coefficient. Sort by absolute value descending. Count how many coefficients are exactly zero vs non-zero, and print the selected features list."
>
> **After running, verify:**
> - DataFrame shows all 8 features sorted by absolute coefficient magnitude
> - Non-zero feature count is reported (Lasso may zero out some features)
> - If feature selection occurred, the list of retained features is printed

In [ ]:
# Extract Lasso coefficients
lasso_coefs = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': lasso_pipeline.named_steps['lasso'].coef_,
    'Abs_Coefficient': np.abs(lasso_pipeline.named_steps['lasso'].coef_)
}).sort_values('Abs_Coefficient', ascending=False)

print("=== LASSO COEFFICIENTS ===")
print(lasso_coefs)

# Count non-zero coefficients
n_nonzero = (lasso_coefs['Coefficient'] != 0).sum()
n_total = len(lasso_coefs)

print(f"\n=== FEATURE SELECTION ===")
print(f"Non-zero features: {n_nonzero} / {n_total}")
print(f"Features zeroed out: {n_total - n_nonzero}")

if n_nonzero < n_total:
    print(f"\n✓ Lasso performed feature selection!")
    print(f"\nSelected features:")
    print(lasso_coefs[lasso_coefs['Coefficient'] != 0]['Feature'].tolist())
else:
    print(f"\n⚠️ All features retained - alpha might be too small")

**Reading the output:**

The coefficient table lists all 8 features with their Lasso-assigned weights, sorted by absolute magnitude. The "Feature Selection" summary at the bottom reports how many coefficients are **non-zero** versus zeroed out. If all 8 remain non-zero, the chosen alpha was relatively light and Lasso kept every feature; if one or more are exactly zero, Lasso has performed automatic feature selection. Either way, compare the ranking to the OLS coefficients from the previous notebook: **MedInc** should still dominate, and features like **AveBedrms** or **Population** may have been shrunk toward (or to) zero.

**Why this matters:** In real-world projects with dozens or hundreds of features, Lasso's ability to discard irrelevant variables can simplify your model dramatically, making it cheaper to deploy and easier to explain to stakeholders.

---

### YOUR FINDINGS:

**Feature Selection:**  
[How many features did Lasso select?]

**Top Features:**  
[Which features have the largest coefficients?]

**Comparison:**  
[How does Lasso performance compare to Ridge?]

---

## 5. Model Comparison Table

With three models fitted (unregularized OLS, Ridge, and Lasso), we bring their key metrics into a single DataFrame for easy comparison. The table includes alpha values, train and validation R², validation MAE, and the number of non-zero features. This side-by-side format makes it straightforward to evaluate the accuracy-complexity tradeoff.
A model that achieves similar R² to OLS but with fewer features or smaller coefficients is preferable for generalization.

> 💡 **Gemini Prompt:** "Fit a plain LinearRegression pipeline (with StandardScaler) as a baseline. Build a comparison DataFrame with columns Model, Alpha, Train_R2, Val_R2, Val_MAE, and Non_Zero_Features for Linear Regression, Ridge, and Lasso. Print the table and identify which model has the best validation R-squared."
>
> **After running, verify:**
> - Three rows appear: Linear Regression, Ridge, and Lasso
> - Val_MAE and Val_R2 allow direct comparison across all three models
> - The best model by Val_R2 is identified at the bottom

In [ ]:
# Baseline for comparison
baseline_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])
baseline_pipeline.fit(X_train, y_train)

comparison = pd.DataFrame([
    {
        'Model': 'Linear Regression',
        'Alpha': 'None',
        'Train_R2': baseline_pipeline.score(X_train, y_train),
        'Val_R2': baseline_pipeline.score(X_val, y_val),
        'Val_MAE': mean_absolute_error(y_val, baseline_pipeline.predict(X_val)),
        'Non_Zero_Features': len(X_train.columns)
    },
    {
        'Model': 'Ridge',
        'Alpha': f"{best_alpha:.4f}",
        'Train_R2': train_score_ridge,
        'Val_R2': val_score_ridge,
        'Val_MAE': mean_absolute_error(y_val, ridge_pipeline.predict(X_val)),
        'Non_Zero_Features': len(X_train.columns)
    },
    {
        'Model': 'Lasso',
        'Alpha': f"{best_alpha_lasso:.4f}",
        'Train_R2': train_score_lasso,
        'Val_R2': val_score_lasso,
        'Val_MAE': mean_absolute_error(y_val, lasso_pipeline.predict(X_val)),
        'Non_Zero_Features': n_nonzero
    }
])

print("=== MODEL COMPARISON ===")
print(comparison.to_string(index=False))

best_model = comparison.loc[comparison['Val_R2'].idxmax(), 'Model']
print(f"\n✓ Best validation performance: {best_model}")

**Reading the output:**

The three-row comparison table shows **Linear Regression** (alpha = None), **Ridge** (with its selected alpha), and **Lasso** (with its selected alpha). Columns include train R², validation R², validation MAE, and the count of non-zero features. On this dataset, all three models likely achieve very similar validation R² (around **0.60**), because the 8-feature problem is simple enough that regularization has limited room to improve. The "Best validation performance" line names the winner, but the practical differences may be within noise. The non-zero-features column is where Lasso's value becomes visible: it achieves comparable accuracy with potentially fewer features.

**Key takeaway:** When models perform similarly, prefer the simpler one. Ridge and Lasso provide insurance against overfitting that will pay off on more complex feature sets, even if the benefit is not dramatic here.

---

## 6. Coefficient Path Visualization

A grouped bar chart comparing the 8 coefficients across all three models reveals the effect of regularization at a glance. Ridge coefficients should be uniformly smaller in absolute value than OLS coefficients (shrinkage), while Lasso coefficients may include exact zeros (sparsity).
This visualization makes the abstract concept of "penalty on coefficient magnitude" concrete and interpretable.

> 💡 **Gemini Prompt:** "Create a grouped bar chart comparing coefficients across Linear Regression, Ridge, and Lasso for all features. Use three bars per feature with different colors, rotated x-labels, a horizontal line at y=0, and a legend. Print observations about how Ridge shrinks coefficients and Lasso can zero some out."
>
> **After running, verify:**
> - Bar chart shows three bars per feature (Linear, Ridge, Lasso side by side)
> - Ridge bars are generally smaller in magnitude than Linear bars
> - Lasso bars may be zero for some features, showing feature selection visually

In [ ]:
# Compare coefficients across models
coef_comparison = pd.DataFrame({
    'Feature': X_train.columns,
    'Linear': baseline_pipeline.named_steps['regressor'].coef_,
    'Ridge': ridge_pipeline.named_steps['ridge'].coef_,
    'Lasso': lasso_pipeline.named_steps['lasso'].coef_
})

# Visualize
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(coef_comparison))
width = 0.25

ax.bar(x - width, coef_comparison['Linear'], width, label='Linear', alpha=0.8)
ax.bar(x, coef_comparison['Ridge'], width, label='Ridge', alpha=0.8)
ax.bar(x + width, coef_comparison['Lasso'], width, label='Lasso', alpha=0.8)

ax.set_xlabel('Features')
ax.set_ylabel('Coefficient Value')
ax.set_title('Coefficient Comparison: Linear vs Ridge vs Lasso')
ax.set_xticks(x)
ax.set_xticklabels(coef_comparison['Feature'], rotation=45, ha='right')
ax.legend()
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
plt.tight_layout()
plt.show()

print("💡 Notice how Ridge shrinks all coefficients")
print("💡 Notice how Lasso can zero out some features completely")

**Reading the output:**

The grouped bar chart displays three bars per feature: blue for **Linear (OLS)**, orange for **Ridge**, and green for **Lasso**. For most features, the Ridge bars are slightly shorter than the OLS bars (uniform shrinkage), while Lasso bars may be noticeably shorter or completely absent for weak predictors. The printed tips below the chart summarize the pattern: Ridge shrinks *all* coefficients, Lasso can *zero out* some entirely.

**Why this matters:** This visualization makes the abstract math of L1 vs. L2 penalties tangible. Stakeholders who cannot parse equations can immediately see that regularization "calms down" large coefficients and that Lasso is more aggressive about discarding features. Use charts like this when presenting model choices to non-technical audiences.

---

## 7. Project Proposal Builder

### Week 1 Milestone: Project Proposal (Due Today)

Your proposal should include:

1. **Dataset Description**
   - Source and link
   - Number of samples and features
   - Brief domain context

2. **Predictive Task**
   - Target variable (what are you predicting?)
   - Prediction unit (what is one row?)
   - Regression or classification?

3. **Evaluation Plan**
   - Primary metric (and why)
   - Train/validation/test split strategy
   - Baseline approach

4. **Leakage Risk Assessment**
   - List 3 potential sources of leakage
   - How you'll prevent them

5. **Initial Concerns**
   - Data quality issues
   - Missing values strategy
   - Class imbalance (if classification)

---

## 📋 PROJECT PROPOSAL TEMPLATE

### 1. Dataset Description

**Dataset Name:**  
[Name]

**Source:**  
[URL or citation]

**Size:**  
[Number of samples × features]

**Domain Context:**  
[Brief description of what the data represents]

---

### 2. Predictive Task

**Target Variable:**  
[What are you predicting?]

**Prediction Unit:**  
[What does one row represent?]

**Task Type:**  
[Regression / Binary Classification / Multiclass Classification]

**Business Question:**  
[Why does this prediction matter?]

---

### 3. Evaluation Plan

**Primary Metric:**  
[MAE / RMSE / R² / Accuracy / F1 / ROC-AUC / etc.]

**Metric Rationale:**  
[Why is this metric aligned with the business goal?]

**Split Strategy:**  
- Training: [%]
- Validation: [%]
- Test: [%]
- Special considerations: [Time-based? Stratified? Grouped?]

**Baseline Approach:**  
[Mean/median predictor, most frequent class, domain heuristic, etc.]

---

### 4. Leakage Risk Assessment

**Risk 1:**  
[Describe potential leakage source and prevention strategy]

**Risk 2:**  
[Describe potential leakage source and prevention strategy]

**Risk 3:**  
[Describe potential leakage source and prevention strategy]

---

### 5. Initial Concerns

**Data Quality:**  
[Issues you've identified or expect]

**Missing Values:**  
[Strategy for handling missingness]

**Class Imbalance (if classification):**  
[How you'll handle it]

**Other Challenges:**  
[Any other concerns]

---

### 6. Success Criteria

**Minimum Viable Model:**  
[What performance would make this useful?]

**Stretch Goal:**  
[What would be excellent performance?]

---

## 8. Wrap-Up: Key Takeaways

### What We Learned Today:

1. **Regularization Intuition**: Penalize complexity to improve generalization
2. **Ridge vs Lasso**: Shrinkage vs sparsity
3. **Cross-Validation Selection**: Let CV choose alpha automatically
4. **Coefficient Interpretation**: Regularization changes coefficient magnitudes
5. **Project Planning**: Clear problem framing prevents wasted effort

### Critical Rules:

> **"Always scale features before regularization"**

> **"Use CV to tune alpha, don't peek at test"**

> **"Start project planning early - dataset choice matters"**

### Next Steps:

- **Submit your project proposal today!**
- Week 2 starts classification (logistic regression, metrics)
- Start collecting your project dataset

---

## Participation Assignment Submission Instructions

### To Submit This Notebook:

1. **Complete all exercises**: Fill in both PAUSE-AND-DO exercise cells with your findings
2. **Run All Cells**: Execute `Runtime → Run all` to ensure everything works
3. **Save a Copy**: `File → Save a copy in Drive or Download the .ipynb extension`
4. **Submit**: Upload your `.ipynb` file in the participation assignment you find in the course Brightspace page.

### Before Submitting, Check:

- [ ] All cells execute without errors
- [ ] All outputs are visible
- [ ] Both exercise responses are complete
- [ ] Notebook is shared with correct permissions
- [ ] You can explain every line of code you wrote

### Next Step:

Complete the **Quiz** in Brightspace (auto-graded)

---

## Bibliography

- James, G., Witten, D., Hastie, T., & Tibshirani, R. (2021). *An Introduction to Statistical Learning with Python* - Linear Model Selection and Regularization
- Hastie, T., Tibshirani, R., & Friedman, J. (2009). *The Elements of Statistical Learning* - Shrinkage and regularization theory
- scikit-learn User Guide: [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html), [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html), [ElasticNet](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html)

---



<center>

Thank you!

</center>